# Crawler de decisões ESAJ
#### Este notebook possui um crawler das decisões do ESAJ de primeira instância.
## Imports e funções

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from selenium import webdriver
from tqdm import tqdm_notebook
re_conteudo = re.compile('(?<=</strong>)(.+)(?=</td>)', flags = re.DOTALL)
re_teor = re.compile('(?<=<span>)(.+)(?=</span>)', flags = re.DOTALL)
re_remhtml = re.compile('<.*?>', flags = re.DOTALL)

def extrair_dados_decisao(decisao): 
    id = classe = assunto = magistrado = comarca = foro = vara = data = None
    id = decisao.find_all('tr')[0].find('span', class_='fonteNegrito').get_text().strip()
    for elemento in decisao.find_all('tr'):
        if elemento.find('strong') != None:
            if elemento.find('strong').get_text().strip() == "Classe:":
                classe = re_conteudo.search(str(elemento))[0].strip()
            elif elemento.find('strong').get_text().strip() == "Assunto:":
                assunto = re_conteudo.search(str(elemento))[0].strip()
            elif elemento.find('strong').get_text().strip() == "Magistrado:":
                magistrado = re_conteudo.search(str(elemento))[0].strip()
            elif elemento.find('strong').get_text().strip() == "Comarca:":
                comarca = re_conteudo.search(str(elemento))[0].strip()
            elif elemento.find('strong').get_text().strip() == "Foro:":
                foro = re_conteudo.search(str(elemento))[0].strip()
            elif elemento.find('strong').get_text().strip() == "Vara:":
                vara = re_conteudo.search(str(elemento))[0].strip()
            elif elemento.find('strong').get_text().strip() == "Data de Disponibilização:":
                data = re_conteudo.search(str(elemento))[0].strip()
        else:
            teor = re_teor.search(str(elemento.find_all('span')[:][1:]))
        
    return((id, classe, assunto, magistrado, comarca, foro, vara, data, re.sub(re_remhtml, '', teor[0])))

def listar_dados_pagina(pagina, driver):
    pagina = str(pagina)
    driver.execute_script("var trocarDePagina = function(pagina){$('div#divDadosResultado').fadeTo('slow', 0.33);$.get(window.saj.env.root+'/trocarDePagina.do', { pagina: pagina}, function(data){atualizarPagina(data)} );};var atualizarPagina = function(data){var div = $('div#resultados');div.html(data);var divResultado = $('div#divDadosResultado', div);divResultado.hide();divResultado.fadeIn('slow');bindPaginacao();var campoPesquisa = $('input[name=dadosConsulta.pesquisaLivre]');bindTermosRelacionados(campoPesquisa);bindImagemMostrarOcultarConteudo();bindLinkDownloadDocumento();};trocarDePagina("+pagina+")")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lista_decisoes_pagina = soup.find_all('tr', class_='fundocinza1')
    lista_dados = [extrair_dados_decisao(decisao) for decisao in lista_decisoes_pagina]
    return lista_dados

### Fazendo a mágica acontecer

In [2]:

driver = webdriver.Firefox(executable_path = 'D:\\programas\\geckodriver\\geckodriver.exe')
driver.get('http://esaj.tjsp.jus.br/cjpg/pesquisar.do?conversationId=&dadosConsulta.pesquisaLivre=JULG*&tipoNumero=UNIFICADO&numeroDigitoAnoUnificado=&foroNumeroUnificado=&dadosConsulta.nuProcesso=&dadosConsulta.nuProcessoAntigo=&classeTreeSelection.values=&classeTreeSelection.text=&assuntoTreeSelection.values=&assuntoTreeSelection.text=&agenteSelectedEntitiesList=&contadoragente=0&contadorMaioragente=0&cdAgente=&nmAgente=&dadosConsulta.dtInicio=&dadosConsulta.dtFim=&varasTreeSelection.values=&varasTreeSelection.text=&dadosConsulta.ordenacao=DESC#')
def extrair_intervalo_pg(lim_inf, lim_sup):
    dados_extraidos = []
    for i in range(lim_inf, lim_sup):
        dados_extraidos = dados_extraidos + listar_dados_pagina(i,driver)
    df = pd.DataFrame(dados_extraidos, columns =['id', 'Classe', 'Assunto', 'Magistrado', 'Comarca', 'Foro', 'Vara', 'Data_disp', 'Teor'])
    df.to_csv('27_10_19_'+str(lim_inf)+'_'+str(lim_sup)+'.csv')

n_ranges = 2914
lim_sup = 1
for i in tqdm_notebook(range(1, n_ranges)):
    lim_inf = lim_sup
    lim_sup = i*500
    extrair_intervalo_pg(lim_inf, lim_sup)

KeyboardInterrupt: 